# Market Microstructure Project

$$
f(S)
\pi_{LB} = Prob \; Liq. \; Buy \\
\pi_{LS} = Prob \; Liq. \; Sell \\
\pi_{I} = Prob \; Informed \; Trade \\
$$

$$
U = \pi_{LB}(A-S_0) + \pi_{LS}(S_0-B)
$$

$$
L = [ \int_{A}^{\infty}f(s)(S-A)dS + \int_{0}^{B}f(s)(B-A)dS ]\pi_I
$$

$$
\max_{B,A}(1-\pi_I)[\pi_{LB}]
$$

DISTR ERLANG TOPA CON .16

CREAR PROPIA FUNCION DE DENSIDAD. WIKIPEDIA -> BUSCAR PDF